In [1]:
import os 

In [4]:
%pwd

'c:\\Dog_vs_Cat_Classifier'

In [3]:
os.chdir('../')

In [14]:
from main.d_vs_c.funtions.comon_funtn import read_yaml, create_directories, save_json_file
from main.d_vs_c.utils import *

class ConfigManeger:
    def __init__(self, config_path=config_file,
                 param_path= params_file):
        self.config = read_yaml(config_path)
        self.parmas = read_yaml(params_file)

        create_directories([self.config.artifacts_folder])

    def get_evalution_config(self):
        config = self.config.trained_model

        create_directories([config.trained_model_path])

        eval_config = {
            'test_dir' : Path(config['test_dir']),
            'trained_model_path' : Path(config['trained_model_path']),
            'model_name' : config['trained_model_name']
        }
        return eval_config



        

In [17]:
import keras
import tensorflow
import mlflow
import dagshub
from mlflow import log_metric, log_param
from mlflow.keras import log_model
from dagshub import DAGsHubLogger

from main.d_vs_c import logger

dagshub.init(repo_owner='sharadnaik001', repo_name='Dog_vs_Cat_Classifier', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)


class Evalution:
    def __init__(self, config, params) -> None:
        self.config = config
        self.params = params

    def laod_nd_eval(self):
        # checking the test data
        test_ds = keras.utils.image_dataset_from_directory(
            directory = self.config['test_dir'],
            image_size = (256, 256)
        )
        # print(test_ds.shape)
        mdl = os.path.join(self.config['trained_model_path'], self.config['model_name'])

        print(f'loading the model..{mdl}')


        self.model = tensorflow.keras.models.load_model(mdl)
        self.model.compile(optimizer = 'adam', loss = 'categorical_crossentropy')
        self.score = self.model.evaluate(test_ds)
        logger.info(f'Printing the Model Performance === > Model Accuracy =:> {self.score[1]} and Model loss =:> {self.score[0]}')
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json_file(path=Path("scores.json"), data=scores)


    def implementing_mlflow(self):
        tracking_uri = 'https://dagshub.com/sharadnaik001/Dog_vs_Cat_Classifier.mlflow'

        mlflow.set_tracking_uri(tracking_uri)
        mlflow.set_experiment("Dog_vs_Cat_Classifier")


        # Step 2: Enable DAGsHub Logger
        dagshub_logger = DAGsHubLogger(metrics_path="metrics.json", hparams_path="hparams.json")
        with mlflow.start_run():
            # logging the model parameters and configure
            log_param('model_name', self.config['model_name'])
            log_param("image_size", (256, 256))
                                             
            # Log metrics: Accuracy and Loss
            log_metric("accuracy", self.score[1])
            log_metric("loss", self.score[0])
      
            # Log the model itself (optional, if retraining)
            mdl = os.path.join(self.config['trained_model_path'], self.config['model_name'])
            self.model = tensorflow.keras.models.load_model(mdl)
            log_model(self.model, "keras_model")      
            # Save the logs to DagsHub
            dagshub_logger.log_metrics({"accuracy": self.score[1],  "loss": self.score[0]})        



[2024-11-23 22:20:35,282: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/sharadnaik001/Dog_vs_Cat_Classifier "HTTP/1.1 200 OK"]
Initialized MLflow to track repo "sharadnaik001/Dog_vs_Cat_Classifier"
[2024-11-23 22:20:35,307: INFO: helpers: Initialized MLflow to track repo "sharadnaik001/Dog_vs_Cat_Classifier"]
Repository sharadnaik001/Dog_vs_Cat_Classifier initialized!
[2024-11-23 22:20:35,321: INFO: helpers: Repository sharadnaik001/Dog_vs_Cat_Classifier initialized!]
[2024-11-23 22:20:35,449: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /sharadnaik001/Dog_vs_Cat_Classifier.mlflow/api/2.0/mlflow/runs/create]
🏃 View run bemused-dove-340 at: https://dagshub.com/sharadnaik001/Dog_vs_Cat_Classifier.mlflow/#/experiments/1/runs/f91b86cb28e346f39448d740979c5c02
🧪 View experiment at: https://dagshub.com/sharadnaik001/Dog_vs_Cat_Cla

In [18]:
obj = ConfigManeger()
confi = obj.get_evalution_config()
obj2 = Evalution(confi, confi)
obj2.laod_nd_eval()
obj2.save_score()
obj2.implementing_mlflow()

{'artifacts_folder': 'artifacts', 'downloading_data': {'source_dir': 'artifacts/raw_data', 'source_URL': 'karakaggle/kaggle-cat-vs-dog-dataset'}, 'preproccessed_data': {'datasets_dir': 'artifacts/datasets', 'train_dir': 'artifacts/datasets/train', 'test_dir': 'artifacts/datasets/test'}, 'image_categories': ['train/Cat', 'train/Dog', 'test/Cat', 'test/Dog'], 'trained_model': {'trained_model_path': 'artifacts/trained_model', 'datasets_dir': 'artifacts/datasets', 'train_dir': 'artifacts/datasets/train', 'test_dir': 'artifacts/datasets/test', 'trained_model_name': 'model_v-03_cpy.h5'}}
{'test_size': 0.2, 'random_state': 42, 'batch_size': 32, 'seed': 128, 'image_size': [256, 256], 'conv_first_layer': 32, 'conv_sec_layer': 64, 'conv_third_layer': 128, 'conv_fourth_layer': 256, 'kernel_size': [3, 3], 'input_shape': [256, 256, 3], 'pool_size': [2, 2], 'strides': 2, 'dropout_rate': 0.25, 'dense_first_layer': 128, 'dense_second_layer': 64, 'output_layer': 1, 'l2_regularization': 0.01, 'optimizer

Found 4970 files belonging to 2 classes.
loading the model..artifacts\trained_model\model_v-03_cpy.h5
[2024-11-23 22:20:54,537: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


c:\Dog_vs_Cat_Classifier\vir_env\Lib\site-packages\keras\src\losses\losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


156/156 ━━━━━━━━━━━━━━━━━━━━ 68s 428ms/step - accuracy: 0.8553 - loss: 0.0823
[2024-11-23 22:22:02,843: INFO: 806364281: Printing the Model Performance === > Model Accuracy =:> 0.8651911616325378 and Model loss =:> 0.08226490020751953]
[2024-11-23 22:22:02,853: INFO: comon_funtn: json file saved at: scores.json]
[2024-11-23 22:22:02,857: INFO: comon_funtn: json file saved at: scores.json]
[2024-11-23 22:22:06,524: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


2024/11/23 22:22:06 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/11/23 22:23:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run chill-elk-392 at: https://dagshub.com/sharadnaik001/Dog_vs_Cat_Classifier.mlflow/#/experiments/1/runs/aee22c79aa224020add7af96593a9393
🧪 View experiment at: https://dagshub.com/sharadnaik001/Dog_vs_Cat_Classifier.mlflow/#/experiments/1


In [ ]:

# export MLFLOW_TRACKING_USERNAME=Sharad Naik
# # export MLFLOW_TRACKING_PASSWORD=<DagsHub-password-or-token>


SyntaxError: invalid syntax (2076406080.py, line 1)